* `Importing modules`

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.models import VGG16_Weights
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

from PIL import Image
import numpy as np
import os
# !pip install torchmetrics
# from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from scipy import linalg
from torchvision import models
from torch import optim
import scipy.io as sio
from tqdm import tqdm
from scipy.ndimage import zoom
import glob
import cv2
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from torchvision.models import vgg16, VGG16_Weights
from torchvision.models.inception import inception_v3, Inception_V3_Weights
import uuid
import matplotlib.pyplot as plt
import torch.optim as optim
from scipy import linalg
import torchvision.transforms.functional as TF
from tqdm import tqdm

# tiny-imagenet-200 dataset paths
COVER_DIR = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "tiny-imagenet-200", "Tiny_imageNet", "val", "images")
SECRET_DIR = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "tiny-imagenet-200", "Tiny_imageNet", "test", "images")
EDGE_DIR_OUTPUT = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "data", "hed_predictions")
EDGE_DIR_OUTPUT_3 = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "data", "hed_predictions_3")
EDGE_DIR_OUTPUT_4 = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "data", "hed_predictions_4")
EDGE_DIR_OUTPUT_5 = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "data", "hed_predictions_5")


# bsds
COVER_DIR_TRAIN = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "BSDS500", "images", "train" )
COVER_DIR_TEST = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "BSDS500", "images", "test")
COVER_DIR_VAL = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "BSDS500", "images", "val" )
EDGE_DIR_TRAIN = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "data", "jpg_edge_images", "train")
gt_dir = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "BSDS500", "groundTruth", "train")

# Model weights
CHECKPOINT_DIR = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "artifacts")
DATA_DIR = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "data")

# GANs
CGANS_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "cGANs_checkpoint_83.pth")
CGANS_OUTPUT_DIR = os.path.join(DATA_DIR, "hed_cGANs")

# CNN
HIDING_NET_PATH = os.path.join(CHECKPOINT_DIR, "hiding_net.pth")
REVEAL_NET_PATH = os.path.join(CHECKPOINT_DIR, "reveal_net.pth")

# Genetics
GENETIC_HED_PATH = os.path.join(CHECKPOINT_DIR, "hed_genetic.pth")
GENETIC_OUTPUT_DIR = os.path.join(os.getcwd(), "drive", "MyDrive", "image steganography", "data", "genetic_predictions")

# CNN results paths
HED_RESULTS = os.path.join(DATA_DIR,  "hed_results")
STEGO_DIR = os.path.join(HED_RESULTS, "CNN_stego")
DECODED_DIR = os.path.join(HED_RESULTS, "CNN_decoded")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# os.makedirs(SECRET_DIR, exist_ok=True)
# os.makedirs(SECRET_DIR2, exist_ok=True)

# os.makedirs(DATA_EDGE_DIR, exist_ok=True)
# os.makedirs(CHECKPOINT_DIR, exist_ok=True)

Using device: cpu


In [ ]:
%%capture
!pip install torchmetrics
!pip install opencv-contrib-python
!git clone https://github.com/s9xie/hed.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def process_hed_with_dataloader(net, data_loader, out_dir, threshold=0.12):
    """
    Process images from a dataloader through HED edge detection model

    Args:
        net: The HED Caffe model loaded with cv2.dnn
        data_loader: DataLoader containing images
        out_dir: Directory to save edge maps
        threshold: Threshold for edge detection
    """
    # Make sure output directory exists
    os.makedirs(out_dir, exist_ok=True)

    # Process each batch
    total_processed = 0
    for batch_data in tqdm(data_loader, desc="Processing HED edges"):
        # Unpack the batch
        images, numeric_ids, img_paths = batch_data

        # Process each image in the batch
        for i in range(len(images)):
            # Convert PyTorch tensor to numpy array for OpenCV
            if torch.is_tensor(images[i]):
                img = images[i].permute(1, 2, 0).numpy()  # Convert from CxHxW to HxWxC
                img = (img * 255).astype(np.uint8)  # Scale from [0,1] to [0,255]
            else:
                # If it's already a PIL Image or numpy array
                img = np.array(images[i])

            # Get original dimensions before any processing
            orig_h, orig_w = img.shape[:2]

            # Resize for HED model
            resized = cv2.resize(img, (321, 481))

            # Create blob for the network
            blob = cv2.dnn.blobFromImage(
                resized,
                scalefactor=1.0,
                size=(321, 481),
                mean=(104.00699, 116.66877, 122.67892),
                swapRB=True,  # convert from cv2's BGR to the RGB ordering used in training (so that caffe sees BGR)
                crop=False
              )

            # Forward pass through the network
            net.setInput(blob)
            hed = net.forward()
            hed = hed[0, 0]  # Shape: (height, width), values in [0, 1]

            # Threshold to keep only strong edges
            hed_binary = (hed > threshold).astype(np.uint8)  # Binary map: 0 or 1

            # Thin the edges using morphological skeletonization
            hed_thinned = cv2.ximgproc.thinning(hed_binary * 255)  # Thinning requires 0 or 255
            hed_thinned = (hed_thinned > 0).astype(np.uint8)  # Back to 0 or 1

            # Resize to original dimensions
            hed_resized = cv2.resize(hed_thinned, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

            # Convert to 0-255 for PNG saving
            hed_final = (hed_resized * 255).astype("uint8")

            # Save edge map using numeric ID if available, otherwise use original filename

            if numeric_ids[i]:
                save_name = f"{numeric_ids[i]}.jpg"
            else:
                # Extract base name without extension
                filename = os.path.splitext(os.path.basename(img_paths[i]))[0]
                save_name = f"{filename}.jpg"

            cv2.imwrite(os.path.join(out_dir, save_name), hed_final)
            total_processed += 1

    print(f"✅ Processed {total_processed} images. All HED outputs saved to: {out_dir}")



In [ ]:
class BSDS500Dataset(Dataset):
    """
    PyTorch Dataset for BSDS500 images (test split).
    Assumes images in <bsds_path>/images/test and ground truths in <bsds_path>/groundTruth/test
    """
    def __init__(self, bsds_path, transform=None):
        self.img_dir = os.path.join(bsds_path)
        # self.gt_dir = os.path.join(bsds_path, 'groundTruth', 'test')
        self.img_paths = sorted(glob(os.path.join(self.img_dir, '*.jpg')))
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, img_path

# # Dataset & Dataloader
# def cv_transform(pil_img):
#     # Resize to 321x481 (width x height) as in HED
#     resized = pil_img.resize((321, 481), Image.BILINEAR)
#     return np.array(resized)

# dataset = BSDS500Dataset(COVER_DIR_TRAIN, transform=cv_transform)
# loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)
# len(dataset)

In [ ]:
def process_with_dataloader(proto, model, bsds_path, out_dir, batch_size=4, use_cuda=False):
    # Load network
    net = cv2.dnn.readNetFromCaffe(proto, model)
    if use_cuda:
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    # Create dataset and dataloader
    transform = transforms.Compose([
        transforms.Resize((481, 321)),  # HED expects height×width as 481×321
        transforms.ToTensor(),
    ])

    dataset = BSDS500Dataset(bsds_path, transform=transform)
    loader = DataLoader(dataset, batch_size, shuffle=False, num_workers=2)

    os.makedirs(out_dir, exist_ok=True)
    saved_paths = []

    # Process
    for batch in tqdm(loader, desc='HED inference'):
        imgs, paths = batch
        for img_tensor, path in zip(imgs, paths):
            # img_np: HxWxC, uint8, RGB
            # Prepare blob: swapRB=True to convert RGB->BGR as Caffe expects

            img_np = img_tensor.permute(1, 2, 0).numpy()
            # Scale from [0, 1] float to [0, 255] uint8
            img_np = (img_np * 255).astype(np.uint8)

            blob = cv2.dnn.blobFromImage(
                img_np,
                scalefactor=1.0,
                size=(321, 481),
                mean=(104.00699, 116.66877, 122.67892),
                swapRB=True,
                crop=False
            )
            net.setInput(blob)
            hed = net.forward()
            # hed shape: (1,1,H,W)
            hed_map = hed[0,0,:,:]
            # Scale to [0,255]
            hed_uint8 = (hed_map * 255).astype(np.uint8)

            # Save as PNG with same basename
            base = os.path.splitext(os.path.basename(path))[0]
            out_path = os.path.join(out_dir, base + '.png')
            cv2.imwrite(out_path, hed_uint8)
            saved_paths.append(out_path)

    return saved_paths

* `Inference`

In [ ]:
# HED model
proto = "/content/hed/examples/hed/deploy.prototxt"
caffe = os.path.join(CHECKPOINT_DIR, "hed_pretrained_bsds.caffemodel")
net = cv2.dnn.readNetFromCaffe(proto, caffe)

# Run inference and save probability maps
print("Starting HED inference...")
saved = process_with_dataloader(
    proto,caffe, COVER_DIR_TRAIN,EDGE_DIR_OUTPUT_3,
    batch_size=4
)
print(f"Saved {len(saved)} edge maps to {EDGE_DIR_OUTPUT_3}")

Starting HED inference...


HED inference: 100%|██████████| 50/50 [05:24<00:00,  6.48s/it]

Saved 200 edge maps to /content/drive/MyDrive/image steganography/data/hed_predictions_3


In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def nms_and_binarize_edges(input_dir, output_dir, threshold=0.2):
    """
    Applies threshold and Non-Maximum Suppression to HED outputs.

    Args:
        input_dir: Path to raw HED output images (float edge maps).
        output_dir: Where to save binarized, NMS-processed edges.
        threshold: Float threshold for binarization (0-1).
    """
    os.makedirs(output_dir, exist_ok=True)

    edge_files = sorted([
        f for f in os.listdir(input_dir)
        if f.endswith('.png') or f.endswith('.jpg')
    ])

    for file in tqdm(edge_files, desc="Applying NMS & binarization"):
        path = os.path.join(input_dir, file)
        edge_map = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        # Normalize if needed
        if edge_map.max() > 1:
            edge_map = edge_map.astype(np.float32) / 255.0

        # Apply threshold
        binary = (edge_map > threshold).astype(np.uint8) * 255

        # Apply thinning (OpenCV's morphological thinning)
        thinned = cv2.ximgproc.thinning(binary)

        # Save result
        save_path = os.path.join(output_dir, file)
        cv2.imwrite(save_path, thinned)



In [ ]:
nms_and_binarize_edges(EDGE_DIR_OUTPUT_3, EDGE_DIR_OUTPUT_5, threshold=0.2)

Applying NMS & binarization: 100%|██████████| 200/200 [00:12<00:00, 15.79it/s]
